In [2]:
import os
import astropy
import numpy as np
from astropy.table import Table
import glob
import matplotlib.pyplot as plt
import pandas as pd
from collections import Counter

In [3]:
directory = r"C:\Users\20xha\Documents\Caltech\Research\SNID\snid_outputs\\"

In [4]:
final_rcf_table = Table.from_pandas(pd.read_hdf("C:/Users/20xha/Documents/Caltech/Research/final_rcf_table.h5"))

In [5]:
SEDM_ML_sample = Table.read("C:/Users/20xha/Documents/Caltech/Research/SEDM_ML_sample.ascii", format = "ascii")

In [6]:
run_list = os.listdir(r"C:\Users\20xha\Documents\Caltech\Research\SNID\snid_outputs/")[3::]

In [15]:
ResultsTable = Table(
                    names=("ZTF_Name", "ZTF_File", "Closest", "Top15", "Good"
                    ),
                    meta={"name": "Spectrum Results after SNID"},
                    dtype=("U64", "U64", "U64", "U64", "U64"
                          )
                    )
count = 0
for i in final_rcf_table["ZTF_Name"]:
    row = []
    row.append(i)
    files = glob.glob(directory + i + "\\*.readableoutput")
    if(len(files) != 0):
        row.append(files[0].split("\\")[-1])
        try:
            templates_list = Table.read(files[0], format = "ascii")
            row.append(templates_list[0]["type"])
            Top15 = templates_list[0:15]
            c = Counter(Top15["type"])
            row.append(c.most_common()[0][0])
            GoodMatches = templates_list[np.where(templates_list["grade"] == "good")]
            c = Counter(GoodMatches["type"])
            row.append(c.most_common()[0][0])
            count += 1
            ResultsTable.add_row(row)
            if count % 100 == 0:
                print(count)
        except:
            print(i)

100
200
ZTF18abntsgz
ZTF18abpaywm
300
ZTF18abvgivy
ZTF18abvmlow
ZTF18acbufud
ZTF18acbvhit
400
500


In [18]:
broken = ["ZTF18abntsgz", "ZTF18abpaywm", "ZTF18abvgivy", "ZTF18abvmlow", "ZTF18acbufud", "ZTF18acbvhit"]

In [19]:
for i in broken:
    row = []
    row.append(i)
    files = glob.glob(directory + i + "\\*.readableoutput")
    if(len(files) != 0):
        row.append(files[0].split("\\")[-1])
        try:
            templates_list = Table.read(files[0], format = "ascii")
            row.append(templates_list[0]["type"])
            Top15 = templates_list[0:15]
            c = Counter(Top15["type"])
            row.append(c.most_common()[0][0])
            GoodMatches = templates_list[np.where(templates_list["grade"] == "good")]
            c = Counter(GoodMatches["type"])
            row.append(c.most_common()[0][0])
            count += 1
            ResultsTable.add_row(row)
            if count % 100 == 0:
                print(count)
        except:
            print(i)

In [26]:
ResultsTable[0]

ZTF_Name,ZTF_File,Closest,Top15,Good
str64,str64,str64,str64,str64
ZTF18aaemivw,ZTF18aaemivw_20181104_P60_v1_snid_templates.readableoutput,Ia-csm,Ia-norm,Ia-norm


In [27]:
final_rcf_table[0]

ZTF_Name,TNS_Name,discovered_by,tns_internal_name,RA_sn,Dec_sn,sn_type,z_sn,g_jd_max,g_max,g_unc_max,r_jd_max,r_max,r_unc_max,ebv,host_name_PS1,RA_host_PS1,Dec_host_PS1,z_host,sep,projected_kpc,gKron_PS1,rKron_PS1,iKron_PS1,zKron_PS1,yKron_PS1,w1_host,w2_host,ebv_host
str12,str9,str13,str29,float64,float64,str9,float64,float64,float64,float64,float64,float64,float64,float64,str21,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
ZTF18aabssth,SN2018aex,ZTF,ZTF18aabssth,165.1891017637735,22.287497041839774,II,0.026000000000000002,2458494.0531481,20.44230842590332,0.1942138969898224,2458218.7148727,18.741601943969727,0.0452471598982811,0.01536,PSO J165.1878+22.2877,165.18778907,22.28771869,0.022914,4.444857426598462,2.0566548608097266,16.885400772094727,16.492399215698242,16.382200241088867,16.378400802612305,16.610000610351562,15.79401524178536,16.38349243564084,0.015369999999999997


In [23]:
JoinedResults = astropy.table.join(ResultsTable, final_rcf_table)

In [34]:
np.unique(JoinedResults["Closest"])

AGN
Gal
II-pec
IIL
IIP
IIb
IIn
Ia-91T
Ia-91bg
Ia-csm
Ia-norm


In [33]:
np.unique(JoinedResults["sn_type"])

II
II-87A
IIb
IIn
Ia
Ia-02cx
Ia-91T
Ia-91bg
Ia-SC
Ib
Ib/c


In [66]:
counters = [0, 0, 0]
closest_list = []
top15_list = []
good_list = []

In [67]:
for i in JoinedResults:
    correct_id = i["sn_type"]
    
    if (correct_id in i["Closest"]):
        counters[0] += 1
    else:
        closest_list.append(np.where(JoinedResults["ZTF_Name"] == i["ZTF_Name"])[0][0])
        
    if (correct_id in i["Top15"]):
        counters[1] += 1
    else:
        top15_list.append(np.where(JoinedResults["ZTF_Name"] == i["ZTF_Name"])[0][0])
        
    if (correct_id in i["Good"]):
        counters[2] += 1
    else:
        good_list.append(np.where(JoinedResults["ZTF_Name"] == i["ZTF_Name"])[0][0])

In [68]:
np.asarray(counters)/(len(JoinedResults))

array([0.82735043, 0.83247863, 0.81025641])

In [69]:
len(JoinedResults)

585

In [82]:
JoinedResults[closest_list]["ZTF_Name", "sn_type", "Closest"].to_pandas().to_csv("SNID_Errors/Closest_Errors.csv")

In [83]:
JoinedResults[top15_list]["ZTF_Name", "sn_type", "Top15"].to_pandas().to_csv("SNID_Errors/Top15_Errors.csv")

In [84]:
JoinedResults[top15_list]["ZTF_Name", "sn_type", "Good"].to_pandas().to_csv("SNID_Errors/Good_Errors.csv")